In [ ]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import itertools 
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [ ]:
folddir_train="/kaggle/input/brain-tumor-mri-dataset/Training"
folders=os.listdir(folddir_train)
train_path=[]
labels=[]
for fold in folders:
    folderpath=os.path.join(folddir_train,fold)
    filepaths=os.listdir(folderpath)
    for path in filepaths:
        imagepaths=os.path.join(folderpath,path)
        train_path.append(imagepaths)
        labels.append(fold) 
train_df=pd.DataFrame(data={"Image Path":train_path,"Label":labels})
train_df


In [ ]:
folddir_test="/kaggle/input/brain-tumor-mri-dataset/Testing"
folds=os.listdir(folddir_test)
image_path=[]
label=[]
for fold in folds:
    folderpath=os.path.join(folddir_test,fold)
    filepaths=os.listdir(folderpath)
    for path in filepaths:
        imagepaths=os.path.join(folderpath,path)
        image_path.append(imagepaths)
        label.append(fold)
dumy_df=pd.DataFrame(data={"File Path":image_path,"Label":label})
dumy_df
                              


In [ ]:
test_df,valid_df =train_test_split(dumy_df,test_size=0.5,random_state=42,stratify=dumy_df["Label"])
test_df

In [ ]:
valid_df

In [ ]:
gen_tr=ImageDataGenerator(rotation_range=0.45,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip=True)
train_gen=gen_tr.flow_from_dataframe(train_df,x_col="Image Path",y_col="Label",target_size=(224,224),color_mode="rgb",class_mode="categorical",patch_size=32)
gen=ImageDataGenerator()
test_gen=gen.flow_from_dataframe(test_df,x_col="File Path",y_col="Label",target_size=(224,224),color_mode="rgb",class_mode="categorical",patch_size=8)
valid_gen=gen.flow_from_dataframe(valid_df,x_col="File Path",y_col="Label",target_size=(224,224),color_mode="rgb",class_mode="categorical",patch_size=8)

In [ ]:
g_dict = train_gen.class_indices      # defines dictionary {'class': index}
classes = list(g_dict.keys())       # defines list of dictionary's kays (classes), classes names : string
images, labels = next(train_gen)      # get a batch size samples from the generator

plt.figure(figsize= (15, 10))

for i in range(16):
    plt.subplot(4, 4, i + 1)
    image =  images[i]/ 255       # scales data to range (0 - 255)
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'blue', fontsize= 12)
    plt.axis('off')

plt.show()

In [ ]:
model=Sequential([
    Conv2D(256,kernel_size=(3,3),activation="relu",input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    Conv2D(256,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(256,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(256,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(256,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(256,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64,activation="relu"),
    Dense(4,activation="softmax")
])
model.summary()

In [ ]:
model.compile(optimizer=Adamax(),loss="categorical_crossentropy",metrics=["accuracy"])


In [ ]:
history=model.fit(train_gen,epochs=100,validation_data=valid_gen,verbose=1)

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')
plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

In [ ]:
train_score = model.evaluate(train_gen, verbose= 1)
test_score = model.evaluate(test_gen, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)

plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation= 45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()

In [ ]:
model.save('Model.h5')

# Test 

In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image
import tensorflow as tf




In [ ]:
loaded_model=load_model("/kaggle/working/Model.h5")
image=Image.open("/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0000.jpg")
image.show()


In [ ]:
img=image.resize((224,224))
img_to_array=tf.keras.preprocessing.image.img_to_array(img)
image_array=tf.expand_dims(img_to_array,0)
pred=loaded_model.predict(image_array)
class_label=[ 'glioma',"meningioma",'notumor', 'pituitary',]
score=tf.nn.softmax(pred[0])
print(class_label[np.argmax(score)])